In [1]:
import requests
import re
import os

url = "https://pic.3gbizhi.com/"
headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Edg/122.0.0.0"
}

response = requests.get(url=url,headers=headers)
response.encoding = response.apparent_encoding

"""
. 表示除空格外任意字符（除\n外）
* 表示匹配字符零次或多次
? 表示匹配字符零次或一次
.*? 非贪婪匹配
"""
parr = re.compile('src="(/u.*?)".alt="(.*?)"') # 匹配图片链接和图片名字
image = re.findall(parr,response.text)

path = "3G壁纸"
if not os.path.isdir(path): # 判断是否存在该文件夹，若不存在则创建
    os.mkdir(path) # 创建
    
# 对列表进行遍历
for i in image:
    link = i[0] # 获取链接
    name = i[1] # 获取名字
    """
    在文件夹下创建一个空jpg文件，打开方式以 'wb' 二进制读写方式
    @param res：图片请求的结果
    """
    with open(path+"/{}.jpg".format(name),"wb") as img:
        res = requests.get("https://pic.3gbizhi.com"+link)
        img.write(res.content) # 将图片请求的结果内容写到jpg文件中
        img.close() # 关闭操作
    print(name+".jpg 获取成功······")


In [1]:
import requests
from lxml import etree
import re
import pymysql
from time import sleep
from concurrent.futures import ThreadPoolExecutor
 
def get_conn():
    # 创建连接
    conn = pymysql.connect(host="127.0.0.1",
                           user="root",
                           password="root",
                           db="novels",
                           charset="utf8")
    # 创建游标
    cursor = conn.cursor()
    return conn, cursor
 
def close_conn(conn, cursor):
    cursor.close()
    conn.close()
 
def get_xpath_resp(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}
    resp = requests.get(url, headers=headers)
    tree = etree.HTML(resp.text)  # 用etree解析html
    return tree,resp
 
def get_chapters(url):
    tree,_ = get_xpath_resp(url)
    # 获取小说名字
    novel_name = tree.xpath('//*[@id="info"]/h1/text()')[0]
    # 获取小说数据节点
    dds = tree.xpath('/html/body/div[4]/dl/dd')
    title_list = []
    link_list = []
    for d in dds[:15]:
        title = d.xpath('./a/text()')[0]  # 章节标题
        title_list.append(title)
        link = d.xpath('./a/@href')[0]   # 章节链接
        chapter_url = url +link  # 构造完整链接
        link_list.append(chapter_url)
    return title_list,link_list,novel_name
 
def get_content(novel_name,title,url):
    try:
        cursor = None
        conn = None
        conn, cursor = get_conn()
        # 插入数据的sql
        sql = 'INSERT INTO novel(novel_name,chapter_name,content) VALUES(%s,%s,%s)'
        tree,resp = get_xpath_resp(url)
        # 获取内容
        content = re.findall('<div id="content">(.*?)</div>',resp.text)[0]
        # 对内容进行清洗
        content = content.replace('<br />','\n').replace('&nbsp;',' ').replace('全本小说网 www.qb5.tw，最快更新<a href="https://www.qb5.tw/book_116659/">宇宙职业选手</a>最新章节！<br><br>','')
        print(title,content)
        cursor.execute(sql,[novel_name,title,content])  # 插入数据
        conn.commit()  # 提交事务保存数据
    except:
        pass
    finally:
        sleep(2)
        close_conn(conn, cursor)  # 关闭数据库
 
 
if __name__ == '__main__':
    # 获取小说名字，标题链接，章节名称
    title_list, link_list, novel_name = get_chapters('https://www.qb5.tw/book_116659/')
    with ThreadPoolExecutor(5) as t:  # 创建5个线程
        for title,link in zip(title_list,link_list):
            t.submit(get_content, novel_name,title,link)  # 启动线程

In [2]:
import re
import urllib.error
import urllib.request

import xlwt
from bs4 import BeautifulSoup


def main():
    baseurl ="http://jshk.com.cn"

    datelist = getDate(baseurl)
    savepath=".\\jshk.xls"
    saveDate(datelist,savepath)

    # askURL("http://jshk.com.cn/")

findlink = re.compile(r'<a href="(.*?)">')
findimg = re.compile(r'<img.*src="(.*?)"',re.S)
findtitle = re.compile(r'<span class="title">(.*)</span')
findrating = re.compile(r'<span class="rating_num" property="v:average">(.*)</span')
findjudge = re.compile(r'<span>(\d*)人评价</span>')
findinq= re.compile(r'<span class="inq">(.*)</span>')

def getDate(baseurl):
    datalist =[]
    for i in range(0,10):
        url=baseurl+str(i*25)
        html=askURL(url)
        soup = BeautifulSoup(html,"html.parser")
        for item in soup.find_all('div',class_="item"):
            data = []
            item = str(item)
            link = re.findall(findlink,item)[0]
            data.append(link)
            img=re.findall(findimg,item)[0]
            data.append(img)
            title=re.findall(findtitle,item)[0]

            rating=re.findall(findrating,item)[0]
            data.append(rating)
            judge=re.findall(findjudge,item)[0]
            data.append(judge)
            inq=re.findall(findinq,item)

            if len(inq)!=0:
                inq=inq[0].replace("。","")
                data.append(inq)
            else:
                data.append(" ")
            print(data)
            datalist.append(data)
        print(datalist)
    return datalist

def askURL(url):
    head = { 
   "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36"}
    request=urllib.request.Request(url,headers=head)
    html=""
    try:
        response=urllib.request.urlopen(request)
        html=response.read().decode("utf-8")
        # print(html)
    except urllib.error.URLError as e:
        if hasattr(e,"code"):
            print(e.code)
        if hasattr(e,"reason"):
            print(e.reason)

    return html

def saveDate(datalist,savepath):
    workbook = xlwt.Workbook(encoding='utf-8')
    worksheet = workbook.add_sheet('电影',cell_overwrite_ok=True)
    col =("电影详情","图片","影片","评分","评价数","概况")
    for i in range(0,5):
        worksheet.write(0,i,col[i])
    for i in range(0,250):
        print("第%d条" %(i+1))
        data=datalist[i]
        for j in range(0,5):
            worksheet.write(i+1,j,data[j])

    workbook.save(savepath)



if __name__ == '__main__':
    main()
    print("爬取完毕")


[Errno 11001] getaddrinfo failed
[]
[Errno 11001] getaddrinfo failed
[]
[Errno 11001] getaddrinfo failed
[]
[Errno 11001] getaddrinfo failed
[]
[Errno 11001] getaddrinfo failed
[]
[Errno 11001] getaddrinfo failed
[]
[Errno 11001] getaddrinfo failed
[]
[Errno 11001] getaddrinfo failed
[]
[Errno 11001] getaddrinfo failed
[]
[Errno 11001] getaddrinfo failed
[]
第1条


IndexError: list index out of range